In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid", context="paper", font_scale=1.4)
plt.rcParams['font.family'] = 'serif'
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['lines.linewidth'] = 2.5
plt.rcParams['figure.figsize'] = (10, 6)

try:
    df = pd.read_csv('benchmark_final_sand_2026.csv', sep=';')
    print("Dataset carregado com sucesso!")
except FileNotFoundError:
    print("ERRO: Arquivo CSV não encontrado.")
    exit()

print("Gerando Figura 1 (Scalability Cost)...")

df_r100 = df[df['Ratio_R'] == 100.0]

fig1_data = df_r100.groupby('TamanhoNode').agg({
    'Cost_Baseline': 'sum',
    'Cost_Temporal': 'sum'
}).reset_index()

fig1_data['Savings_Pct'] = ((fig1_data['Cost_Baseline'] - fig1_data['Cost_Temporal']) / fig1_data['Cost_Baseline']) * 100

plt.figure()
ax = sns.lineplot(data=fig1_data, x='TamanhoNode', y='Savings_Pct', 
                  marker='o', markersize=10, color='#2c3e50')

plt.title('Cost Savings Scalability (Critical Scenario R=100)')
plt.xlabel('Cluster Size (Number of Nodes)')
plt.ylabel('Average Cost Reduction (%)')
plt.ylim(0, 80)
plt.xticks([10, 20, 50, 100, 200])
plt.grid(True, linestyle='--', alpha=0.7)

for x, y in zip(fig1_data['TamanhoNode'], fig1_data['Savings_Pct']):
    plt.text(x, y + 2.5, f'{y:.1f}%', ha='center', va='bottom', fontweight='bold', fontsize=12)

plt.tight_layout()
plt.savefig('fig1.png', dpi=300)
plt.close()

print("Gerando Figura 2 (Stability/Volatility)...")

mask = (df['Ratio_R'] == 100.0) & (df['TamanhoNode'] == 50) & (df['TamanhoPod'] == 1000) & (df['Execucao'] == 1)
df_fig2 = df[mask].sort_values('Slot')

if not df_fig2.empty:
    plt.figure()
    
    plt.plot(df_fig2['Slot'], df_fig2['Cost_Baseline'], label='Baseline (Reactive)', 
             color='#c0392b', marker='o', markersize=6)
    
    plt.plot(df_fig2['Slot'], df_fig2['Cost_Temporal'], label='Temporal (TGCH)', 
             color='#27ae60', marker='s', markersize=6)
    
    plt.fill_between(df_fig2['Slot'], df_fig2['Cost_Baseline'], df_fig2['Cost_Temporal'], 
                     color='gray', alpha=0.15, label='Cost Avoidance')

    plt.title('Operational Stability: Cost Evolution Over Time')
    plt.xlabel('Time Slot (Planning Horizon)')
    plt.ylabel('Total Cost ($)')
    plt.legend(loc='upper right', frameon=True)
    plt.xticks(range(1, 21, 2))
    
    plt.tight_layout()
    plt.savefig('fig2.png', dpi=300)
    plt.close()

print("Gerando Figura 3 (Execution Time Scalability)...")

fig3_data = df.groupby('TamanhoNode').agg({
    'Time_Baseline_ms': 'mean',
    'Time_Temporal_ms': 'mean'
}).reset_index()

fig3_melt = fig3_data.melt(id_vars=['TamanhoNode'], 
                           value_vars=['Time_Baseline_ms', 'Time_Temporal_ms'],
                           var_name='Algorithm', value_name='Time_ms')

fig3_melt['Algorithm'] = fig3_melt['Algorithm'].replace({
    'Time_Baseline_ms': 'Baseline (Reactive)',
    'Time_Temporal_ms': 'Temporal (TGCH)'
})

plt.figure()
sns.lineplot(data=fig3_melt, x='TamanhoNode', y='Time_ms', hue='Algorithm', style='Algorithm',
             markers=True, markersize=9, dashes=False)

plt.title('Algorithm Scalability: Decision Latency vs Cluster Size')
plt.xlabel('Cluster Size (Number of Nodes)')
plt.ylabel('Avg Decision Time (ms)')
plt.legend(title=None)
plt.xticks([10, 20, 50, 100, 200])

plt.tight_layout()
plt.savefig('fig_scalability_time.png', dpi=300)
plt.close()

print("\n--- SUCESSO! ---")
print("Geradas: fig1.png, fig2.png, fig_scalability_time.png")